## Model building: Latent Dirichlet Allocation

Description: 

In section 1) Model tuning (hyperparameter selection) - decide on the number of topics (n_components) to use in the model. Run 5-fold cross validation for n_components in range (2-30). Use perplexity calculated on the test set as the evaluation criteria

In section 2) train full model with optimal number of topics determined from section 1.


LDA code was adapted from: [scikit_learn](https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py) - Authors: Olivier Grisel, Lars Buitinck, Chyi-Kwei Yau - License: BSD 3 clause

In [3]:
import pandas as pd
import numpy as np
from scipy import sparse
import time
import pickle

from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

## 1) Hyperparameter selection - determine number of topcs (n_components)

In [8]:
# read in scipy sparse matrix
X = sparse.load_npz("data_preprocessing_out\\X.npz")

with open("data_preprocessing_out\\word_key.txt", "rb") as f:
    word_key = pickle.load(f)

n_movies, n_features = X.shape


In [ ]:
# no. of documents for train/test data
n_train_samples = 8000
n_test_samples = 2000

# hyperparameter selection - number of topics (n_components)
n_folds = 5
rand_seeds = [0, 11, 22, 33, 44]
n_components = np.array(
    [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
)
perp_train = np.zeros((n_components.shape[0], n_folds))
perp_test = np.zeros((n_components.shape[0], n_folds))
for k in range(n_folds):
    # construct train/test split
    print(f"fold number {k}:")
    X_train, X_test = train_test_split(
        X,
        train_size=n_train_samples,
        test_size=n_test_samples,
        random_state=rand_seeds[k],
    )
    for j, ncomp in enumerate(n_components):
        lda = LatentDirichletAllocation(
            n_components=ncomp,  # x4 components 2.5x the time
            max_iter=55,  # x2 iterations 2x the time
            learning_decay=0.55,
            learning_method="online",
            learning_offset=64.0,
            batch_size=256,
            random_state=0,
            evaluate_every=5,
            verbose=0,
            n_jobs=-1,
        )
        lda.fit(X_train)  # scales (roughly) linearly with number of training examples
        perp_train[j, k] = np.round(lda.perplexity(X_train), 1)
        perp_test[j, k] = np.round(lda.perplexity(X_test), 1)
        if k == 0:
            pickle.dump(
                lda,
                open(
                    "model_building\\hyperparameter_tuning\\model_"
                    + str(ncomp)
                    + "_components.sav",
                    "wb",
                ),
            )
        print(
            f"n_components = {ncomp}: p_train = {perp_train[j, k]}, p_test = {perp_test[j, k]}"
        )

with open("model_building\\perp_train.txt", "wb") as f:
    pickle.dump(perp_train, f)
with open("model_building\\perp_test.txt", "wb") as f:
    pickle.dump(perp_test, f)

In [10]:
# compute mean and std pf perplexity on the train/test data
perp_test_mean = np.mean(perp_test, axis=1)
perp_test_std = np.std(perp_test, axis=1)

perp_train_mean = np.mean(perp_train, axis=1)
perp_train_std = np.std(perp_train, axis=1)

In [ ]:
# plot the mean perplexity for train/test data as
# a function of n_components

plt.rc("font", **{"size": 14})

fig, ax = plt.subplots(1, 1)
plt.plot(
    np.linspace(10, 30.5, 100),
    np.mean(perp_test_mean[-4:]) * np.ones(shape=(100,)),
    "k--",
)
plt.errorbar(
    n_components,
    perp_test_mean[:],
    yerr=perp_test_std,
    xerr=None,
    fmt="r*",
    label="test",
)
plt.errorbar(
    n_components,
    perp_train_mean[:],
    yerr=perp_train_std,
    xerr=None,
    fmt="b*",
    label="train",
)

ax.set_ylabel("perplexity")
ax.set_xlabel("n_components")
plt.legend()

## 2) Fit full model with predetermined n_components

In [ ]:
# fit model with optimal value for n_components

# model inputs
n_samples = 12000
n_components = 20
n_top_words = 20

# construct train/test split
X_train, X_test = train_test_split(
    X, train_size=n_samples, test_size=1000, random_state=22
)

# took 90 minutes (~1.75 minutes/iteration) on 10,000 words, 20 components, 10177 randomly chosen samples, 52 iterations, perplexity dropped <0.1/iteration at 52th iteration
print(
    f"Fitting LDA models with tf features, n_samples={n_samples} and n_features={n_features}..."
)
lda = LatentDirichletAllocation(
    n_components=n_components,  # x4 components 2.5x the time
    max_iter=30,  # x2 iterations 2x the time
    learning_decay=0.55,
    learning_method="online",
    learning_offset=64.0,
    batch_size=256,
    random_state=0,
    evaluate_every=5,
    verbose=1,
    n_jobs=-1,
)
t0 = time.time()
lda.fit(X_train)  # scales (roughly) linearly with number of training examples
print("done in %0.3fs." % (time.time() - t0))

print(lda.perplexity(X_test))

# predict topic weights for all scripts in the database (4 minutes for the entire corpus)
# this will be precomputed and saved to be used for recommender app predicitons
Xtran = lda.transform(X)
with open("model_building\\Xtran.txt", "wb") as f:
    pickle.dump(Xtran, f)
pickle.dump(lda, open("model_building\\model_2023_08_24.sav", "wb"))